<a href="https://colab.research.google.com/github/marlongabrielk/Aniversariometro/blob/main/Aniversariometro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# CÉLULA 1: APENAS PARA MONTAR O GOOGLE DRIVE E CONFIGURAR CAMINHOS
# IMPORTANTE: Execute esta célula primeiro e complete o processo de autorização do Google Drive
# antes de executar a Célula 2.

# Importa a funcionalidade para interagir com o Google Drive no ambiente Colab.
from google.colab import drive
# Importa o módulo 'os' para manipulação de caminhos de arquivos e diretórios
# de forma compatível com o sistema operacional.
import os

# --- Variáveis Globais para Configuração do Drive e Arquivo ---
# Estas variáveis são declaradas como "globais" (neste contexto do notebook) para que
# a Célula 2 possa acessá-las e saber se o Drive foi montado corretamente e onde
# encontrar/salvar o arquivo de dados.

# Flag para indicar se a montagem do Drive foi bem-sucedida. Inicia como False.
drive_montado_com_sucesso_global = False
# Variável para armazenar o caminho completo do arquivo JSON onde os dados serão salvos. Inicia como None.
NOME_ARQUIVO_DADOS_GLOBAL = None
# Variável para armazenar o caminho do diretório no Drive onde a aplicação guardará seus dados. Inicia como None.
DRIVE_DATA_PATH_GLOBAL = None

print("🚀 Iniciando montagem do Google Drive...")
print("   Por favor, siga as instruções na tela para autorizar o acesso ao seu Google Drive.")

try:
    # Tenta montar o Google Drive.
    # '/content/drive' é o ponto padrão onde o Drive é montado no sistema de arquivos do Colab.
    # force_remount=True força o Colab a pedir autorização toda vez. Isso é útil para
    # garantir que está usando a conta correta e para scripts compartilhados.
    drive.mount('/content/drive', force_remount=True)
    print("\n✅ Google Drive montado com sucesso em '/content/drive'!")

    # --- Define os caminhos para os dados da aplicação no Google Drive ---
    # Nome da pasta que será criada/usada no seu "Meu Drive" para esta aplicação.
    # Você pode alterar este nome se preferir.
    APP_DATA_FOLDER_NAME_GLOBAL = "AniversariometroApp_Dados_Final"
    # Caminho base para o "Meu Drive" no ambiente Colab.
    DRIVE_BASE_PATH_GLOBAL = "/content/drive/MyDrive"

    # Cria o caminho completo para a pasta da aplicação usando os.path.join.
    # os.path.join junta os nomes de forma inteligente, criando um caminho válido.
    DRIVE_DATA_PATH_GLOBAL = os.path.join(DRIVE_BASE_PATH_GLOBAL, APP_DATA_FOLDER_NAME_GLOBAL)
    # Define o nome do arquivo JSON onde os dados dos aniversários serão guardados.
    # Você pode alterar "aniversarios_gdrive_final.json" se desejar.
    NOME_ARQUIVO_DADOS_GLOBAL = os.path.join(DRIVE_DATA_PATH_GLOBAL, "aniversarios_gdrive_final.json")

    print(f"📂 O diretório da aplicação no seu Google Drive será: {DRIVE_DATA_PATH_GLOBAL}")
    print(f"📄 O arquivo de dados dos aniversários será: {NOME_ARQUIVO_DADOS_GLOBAL}")

    # --- Cria o diretório da aplicação no Drive se ele ainda não existir ---
    # os.path.exists verifica se o caminho (pasta) já existe.
    if not os.path.exists(DRIVE_DATA_PATH_GLOBAL):
        # os.makedirs cria o diretório.
        # exist_ok=True faz com que a função não gere um erro se o diretório já existir.
        os.makedirs(DRIVE_DATA_PATH_GLOBAL, exist_ok=True)
        print(f"👍 Diretório '{APP_DATA_FOLDER_NAME_GLOBAL}' criado/verificado com sucesso no seu Google Drive.")
    else:
        print(f"👍 Diretório '{APP_DATA_FOLDER_NAME_GLOBAL}' já existe no seu Google Drive.")

    # Se tudo deu certo até aqui, atualiza a flag global.
    drive_montado_com_sucesso_global = True
    print("\n🎉 Configuração do Google Drive finalizada com sucesso!")
    print("   Agora você pode executar a CÉLULA 2 com o código principal do Aniversariômetro.")

except Exception as e:
    # Se qualquer erro ocorrer durante a montagem ou configuração dos caminhos:
    print(f"\n🚨🚨 FALHA CRÍTICA AO MONTAR OU CONFIGURAR O GOOGLE DRIVE: {e} 🚨🚨")
    print("   Por favor, verifique as permissões, o espaço no Drive e tente executar esta célula novamente.")
    print("   O programa principal (CÉLULA 2) não poderá gerenciar dados de aniversários sem o Drive.")
    # As variáveis globais permanecem com seus valores iniciais (False, None),
    # indicando para a Célula 2 que houve um problema.

🚀 Iniciando montagem do Google Drive...
   Por favor, siga as instruções na tela para autorizar o acesso ao seu Google Drive.
Mounted at /content/drive

✅ Google Drive montado com sucesso em '/content/drive'!
📂 O diretório da aplicação no seu Google Drive será: /content/drive/MyDrive/AniversariometroApp_Dados_Final
📄 O arquivo de dados dos aniversários será: /content/drive/MyDrive/AniversariometroApp_Dados_Final/aniversarios_gdrive_final.json
👍 Diretório 'AniversariometroApp_Dados_Final' criado/verificado com sucesso no seu Google Drive.

🎉 Configuração do Google Drive finalizada com sucesso!
   Agora você pode executar a CÉLULA 2 com o código principal do Aniversariômetro.


In [ ]:
# CÉLULA 2: CÓDIGO PRINCIPAL DO ANIVERSARIÔMETRO INTELIGENTE (VERSÃO FINAL COMENTADA)
# IMPORTANTE: Execute esta célula SOMENTE APÓS a CÉLULA 1 ter sido executada com sucesso
# e o Google Drive ter sido montado corretamente.

# Importa a biblioteca do Google para interagir com os modelos Generative AI (Gemini).
import google.generativeai as genai
# Importa as classes 'datetime' e 'timedelta' para trabalhar com datas e durações.
from datetime import datetime, timedelta
# Importa 'userdata' do Colab para buscar a API Key armazenada de forma segura nos "Secrets".
from google.colab import userdata
# Importa o módulo 'json' para trabalhar com arquivos no formato JSON (para salvar/carregar dados).
import json
# Importa 'os' novamente, pois pode ser usado em funções aqui também (ex: os.path.exists).
import os

# --- Instalação da biblioteca do Gemini ---
# Este comando instala a biblioteca 'google-generativeai' no ambiente do Colab,
# caso ela ainda não esteja instalada. O '-q' é para "quiet" (menos mensagens durante a instalação).
print("📦 Instalando/verificando biblioteca google-generativeai...")
!pip install -q google-generativeai
print("   Biblioteca google-generativeai pronta.")

# --- Configuração da API Key e Modelo Gemini ---
# Esta seção depende das variáveis globais 'drive_montado_com_sucesso_global' e
# 'NOME_ARQUIVO_DADOS_GLOBAL' que foram (ou deveriam ter sido) definidas pela CÉLULA 1.

model = None # Inicializa a variável 'model' como None. Ela guardará o modelo de IA.
try:
    # Verifica se as variáveis globais da Célula 1 existem.
    # Isso é uma proteção para o caso de esta célula ser executada acidentalmente antes da Célula 1.
    if 'drive_montado_com_sucesso_global' not in globals() or \
       'NOME_ARQUIVO_DADOS_GLOBAL' not in globals():
        # Se as variáveis não existem, lança um erro para interromper a execução.
        raise NameError("Dependências da Célula 1 (montagem do Drive) não encontradas. Execute a Célula 1 primeiro.")

    # Configura a API Key do Gemini.
    # userdata.get('GEMINI_API_KEY') busca a chave que você deve ter configurado
    # nos "Secrets" (segredos) do Google Colab com o nome 'GEMINI_API_KEY'.
    genai.configure(api_key=userdata.get('GEMINI_API_KEY'))
    # Inicializa o modelo Generative AI.
    # 'gemini-1.5-flash-latest' é um modelo rápido e eficiente.
    model = genai.GenerativeModel('gemini-1.5-flash-latest')
    print("🔑 API Key do Gemini configurada e modelo 'gemini-1.5-flash-latest' carregado!")
except NameError as ne:
    # Captura o erro específico se as variáveis da Célula 1 não foram encontradas.
    print(f"🔴 ERRO DE CONFIGURAÇÃO: {ne}")
    print("   Por favor, execute a CÉLULA 1 para montar o Google Drive antes de executar esta célula.")
except Exception as e:
    # Captura outros possíveis erros durante a configuração da API ou do modelo.
    print(f"🚨 Erro ao configurar a API Key ou carregar o modelo Gemini: {e}")
    print("   Verifique se você configurou o segredo 'GEMINI_API_KEY' nos Secrets do Colab.")
    # 'model' permanecerá None se houver erro.

# --- Funções para Persistência de Dados (Salvar/Carregar) ---
def carregar_aniversarios_do_arquivo(nome_arquivo_ref, drive_ok_ref):
    """
    Carrega a lista de aniversários de um arquivo JSON no Google Drive.
    Se o arquivo não existir, cria um novo com um exemplo.
    Retorna uma lista vazia se o Drive não estiver acessível ou ocorrer um erro crítico.
    Parâmetros:
        nome_arquivo_ref: O caminho completo do arquivo JSON (vem da Célula 1).
        drive_ok_ref: A flag booleana indicando se o Drive foi montado (vem da Célula 1).
    """
    # Lista de exemplo para usar se o arquivo JSON não existir ou estiver corrompido.
    base_exemplos = [
        {"nome": "Exemplo Aniversariante", "data_nasc": "2025-12-25", "interesses": ["programação", "IA", "café"], "restricoes": [], "relacionamento": "inspiração"},
    ]

    # Verifica se o Drive está OK e se o nome do arquivo foi definido.
    if not drive_ok_ref or not nome_arquivo_ref:
        print("❌ Google Drive não acessível ou caminho do arquivo não definido (problema na Célula 1). Não é possível carregar aniversários.")
        return [] # Retorna uma lista vazia para evitar erros.

    # O diretório pai (ex: AniversariometroApp_Dados_Final) já deve ter sido criado pela Célula 1.
    # Aqui, apenas verificamos se o ARQUIVO JSON em si existe.
    if not os.path.exists(nome_arquivo_ref):
        print(f"ℹ️ Arquivo '{nome_arquivo_ref}' não encontrado. Criando com 1 exemplo.")
        # Se o arquivo não existe, chama a função para salvá-lo com os dados de exemplo.
        salvar_aniversarios_no_arquivo(nome_arquivo_ref, base_exemplos, drive_ok_ref)
        return base_exemplos # Retorna a lista de exemplos.

    try:
        # Tenta abrir o arquivo JSON para leitura ('r').
        # 'encoding='utf-8'' é importante para suportar caracteres especiais.
        with open(nome_arquivo_ref, 'r', encoding='utf-8') as f:
            # json.load(f) lê o conteúdo do arquivo JSON e o converte para uma lista Python.
            dados = json.load(f)
        print(f"✅ Aniversários carregados de '{nome_arquivo_ref}' com sucesso!")
        return dados
    except (json.JSONDecodeError, IOError) as e:
        # Se houver erro ao ler o arquivo (ex: JSON mal formatado, problema de I/O):
        print(f"⚠️ Erro ao carregar ou decodificar o arquivo JSON '{nome_arquivo_ref}': {e}.")
        print("   Tentando recriar o arquivo com o exemplo padrão para restaurar a funcionalidade.")
        # Tenta salvar o arquivo com os exemplos para que o programa possa continuar em uma base limpa.
        salvar_aniversarios_no_arquivo(nome_arquivo_ref, base_exemplos, drive_ok_ref)
        return base_exemplos # Retorna a lista de exemplos como fallback.

def salvar_aniversarios_no_arquivo(nome_arquivo_ref, dados_aniversarios, drive_ok_ref):
    """
    Salva a lista de aniversários (que está na memória) no arquivo JSON no Google Drive.
    Parâmetros:
        nome_arquivo_ref: O caminho completo do arquivo JSON.
        dados_aniversarios: A lista Python de dicionários de aniversariantes.
        drive_ok_ref: A flag booleana indicando se o Drive foi montado.
    """
    if not drive_ok_ref or not nome_arquivo_ref:
        print("❌ Google Drive não acessível ou caminho do arquivo não definido (problema na Célula 1). Dados NÃO foram salvos.")
        return

    try:
        # O diretório pai já foi criado/verificado pela Célula 1.
        # Abre o arquivo JSON para escrita ('w'). Isso sobrescreve o arquivo existente.
        with open(nome_arquivo_ref, 'w', encoding='utf-8') as f:
            # json.dump() escreve a lista Python no arquivo em formato JSON.
            # indent=4 formata o JSON de forma legível (com indentação).
            # ensure_ascii=False permite que caracteres acentuados sejam salvos corretamente.
            json.dump(dados_aniversarios, f, indent=4, ensure_ascii=False)
        # print(f"💾 Aniversários salvos em '{nome_arquivo_ref}'") # Comentado para não poluir a saída normal.
    except (IOError, OSError) as e:
        # Captura erros de escrita no arquivo ou problemas de acesso ao diretório.
        print(f"❌❌ ERRO CRÍTICO ao salvar aniversários em '{nome_arquivo_ref}': {e}")
        print("     Verifique as permissões do Google Drive e o espaço disponível. Os dados podem NÃO ter sido salvos.")

# --- Carrega a base de dados ---
# A lista 'aniversarios_cadastrados' guardará todos os aniversários na memória do programa.
aniversarios_cadastrados = [] # Inicializa como lista vazia.
# Verifica se as flags da Célula 1 indicam que o Drive está OK.
if 'drive_montado_com_sucesso_global' in globals() and drive_montado_com_sucesso_global:
    # Se o Drive estiver OK, carrega os aniversários do arquivo.
    aniversarios_cadastrados = carregar_aniversarios_do_arquivo(NOME_ARQUIVO_DADOS_GLOBAL, drive_montado_com_sucesso_global)
else:
    # Se houve problema na Célula 1, informa o usuário.
    print("⚠️ Aniversários não puderam ser carregados (Google Drive não foi montado corretamente na Célula 1).")


# --- Agente 1: "Guardião das Datas" ---
def guardar_aniversario(nome, data_str, interesses, restricoes, relacionamento):
    """Adiciona um novo aniversário à lista em memória e depois salva toda a lista no arquivo JSON."""
    # Verifica se o Drive está OK (flag da Célula 1).
    if 'drive_montado_com_sucesso_global' not in globals() or not drive_montado_com_sucesso_global:
        print("🔴 Drive não montado (problema na Célula 1). Não é possível guardar aniversário.")
        return

    # Verifica se já existe um aniversariante com o mesmo nome e data para evitar duplicatas simples.
    for aniv in aniversarios_cadastrados:
        if aniv['nome'].lower() == nome.lower() and aniv['data_nasc'] == data_str:
            print(f"ℹ️ {nome} ({data_str}) já está cadastrado(a). Nenhuma alteração feita.")
            return # Sai da função se já existe.

    # Adiciona o novo aniversário (como um dicionário) à lista 'aniversarios_cadastrados' que está na memória.
    aniversarios_cadastrados.append({
        "nome": nome, "data_nasc": data_str, "interesses": interesses,
        "restricoes": restricoes, "relacionamento": relacionamento
    })
    # Após adicionar à memória, salva a lista COMPLETA e ATUALIZADA no arquivo JSON no Drive.
    salvar_aniversarios_no_arquivo(NOME_ARQUIVO_DADOS_GLOBAL, aniversarios_cadastrados, drive_montado_com_sucesso_global)
    print(f"🎉 Aniversário de {nome} ({data_str}) cadastrado e salvo em '{NOME_ARQUIVO_DADOS_GLOBAL}'!")

def verificar_aniversarios_proximos(dias_antecedencia=7):
    """
    Verifica aniversários próximos usando a data/hora atual ou uma data simulada (se descomentada).
    Retorna uma lista de dicionários dos aniversariantes próximos.
    """
    # --- DETERMINAÇÃO DA DATA "HOJE" ---
    # Por padrão, usa a data e hora atuais do sistema onde o Colab está rodando.
    hoje_obj = datetime.now()
    # A linha abaixo imprime qual data está sendo usada (a real ou a simulada).
    # É útil para saber, mas pode ser comentada para uma saída mais limpa.
    # print(f"ℹ️ Usando data/hora ATUAL para verificação: {hoje_obj.strftime('%Y-%m-%d %H:%M:%S')}")

    # --- PARA TESTES: SIMULAÇÃO DE DATA "HOJE" ---
    # Se você precisar testar o comportamento do programa para uma data específica (ex: para ver se
    # um aniversário cadastrado para "amanhã" realmente aparece), você pode DESCOMENTAR a linha abaixo
    # e COMENTAR a linha "hoje_obj = datetime.now()" acima.
    # Exemplo: Para simular que "hoje" é 17 de Maio de 2025, às 10:00 da manhã:
    # hoje_obj = datetime(2025, 5, 17, 10, 0, 0)
    # Se a linha de simulação estiver ativa, o print abaixo dará um feedback.
    # (Este if apenas checa se a variável 'hoje_obj' foi definida pela simulação ou pelo datetime.now())
    # if hoje_obj.replace(microsecond=0) != datetime.now().replace(microsecond=0, second=0, minute=0, hour=0): # Compara só a data
    #    print(f"⚠️ ATENÇÃO: Usando data SIMULADA para verificação: {hoje_obj.strftime('%Y-%m-%d %H:%M:%S')} ⚠️")


    hoje_data = hoje_obj.date() # Extrai apenas a parte da DATA (ano, mês, dia) de 'hoje_obj'.
    alertas = [] # Lista para armazenar os aniversariantes próximos.

    # Itera sobre cada aniversariante na lista 'aniversarios_cadastrados' (que está na memória).
    for aniv in aniversarios_cadastrados:
        data_aniv_str = aniv["data_nasc"] # Pega a data de nascimento (string) do cadastro.
        try:
            # Converte a string da data de nascimento para um objeto datetime.
            data_aniv_obj_original = datetime.strptime(data_aniv_str, "%Y-%m-%d")
        except ValueError:
            # Se a data no cadastro não estiver no formato "AAAA-MM-DD", avisa e pula este aniversariante.
            print(f"⚠️ Aviso: Formato de data inválido para '{aniv['nome']}' ('{data_aniv_str}'). Use AAAA-MM-DD. Pulando.")
            continue # Vai para o próximo aniversariante no loop.

        # Calcula a data do aniversário para o ANO CORRENTE.
        # Ex: Se nasceu em "1990-07-25" e hoje_data.year é 2025, isso vira "2025-07-25".
        data_aniv_para_comparar = data_aniv_obj_original.replace(year=hoje_data.year).date()

        # Verifica se o aniversário NO ANO CORRENTE já passou em relação à data de "hoje".
        if data_aniv_para_comparar < hoje_data:
            # Se já passou, ajusta a data do aniversário para o PRÓXIMO ANO.
            # Ex: Se "hoje" é 2025-10-15 e o aniversário é 2025-07-25 (já passou),
            #     então considera 2026-07-25 para a próxima ocorrência.
            data_aniv_para_comparar = data_aniv_para_comparar.replace(year=hoje_data.year + 1)

        # Calcula a diferença em dias entre a próxima data do aniversário e "hoje".
        diferenca_em_dias = (data_aniv_para_comparar - hoje_data).days

        # Verifica se a diferença está dentro do intervalo desejado:
        # 0 dias = aniversário é hoje.
        # dias_antecedencia = até quantos dias no futuro queremos o alerta.
        if 0 <= diferenca_em_dias <= dias_antecedencia:
            # Se a condição for atendida, adiciona o aniversariante à lista de alertas.
            aniv_com_info = aniv.copy() # Cria uma cópia para não modificar o original na lista principal.
            aniv_com_info['dias_para_aniversario'] = diferenca_em_dias # Guarda a diferença de dias.
            alertas.append(aniv_com_info)

    # Ordena os alertas pela proximidade do aniversário (quem está mais perto aparece primeiro).
    alertas.sort(key=lambda x: x['dias_para_aniversario'])
    return alertas

# --- Agente 2: "Mago dos Parabéns" ---
def gerar_mensagem_parabens(nome, relacionamento, dias_para_aniversario):
    """Gera uma mensagem de parabéns personalizada usando o modelo Gemini Flash."""
    if not model: return "Modelo Gemini não configurado (verifique API Key e Célula 1)."
    # Adapta a urgência da mensagem se o aniversário for hoje.
    urgencia = "HOJE MESMO! 🥳" if dias_para_aniversario == 0 else f"daqui a {dias_para_aniversario} dia(s)!"
    # Cria o prompt para o modelo de IA.
    prompt = f"Crie uma mensagem de parabéns curta (2-4 frases), carinhosa e criativa para {nome}. O aniversário de {nome} é {urgencia}. Meu relacionamento com essa pessoa é: {relacionamento}. Adapte o tom (Ex: amiga próxima, colega de trabalho formal, irmão querido). Ideal para enviar por WhatsApp. Seja original, demonstre afeto/consideração. Finalize com um emoji festivo."
    try:
        # Envia o prompt para o modelo Gemini e retorna o texto da resposta.
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        return f"Erro ao gerar mensagem com Gemini: {e}"

# --- Agente 3: "Detetive de Presentes" ---
def simular_catalogo_presentes(interesse_chave):
    """Simula um catálogo de ideias de presentes baseadas em interesses."""
    # Este é um catálogo simplificado. Em uma aplicação real, poderia vir de um banco de dados
    # ou de uma API de e-commerce, por exemplo.
    catalogo = {
        "programação": ["livro de algoritmos avançados", "curso de nova linguagem", "teclado mecânico ergonômico"],
        "ia": ["livro sobre ética em IA", "assinatura de revista de tecnologia com foco em IA", "curso online de machine learning"],
        "café": ["kit de degustação de cafés especiais de diferentes origens", "moedor de café manual de precisão", "caneca térmica inteligente com controle de temperatura"],
        "jardinagem": ["kit de ferramentas de jardinagem premium em aço inoxidável", "vaso autoirrigável estiloso para plantas internas", "livro ilustrado sobre design de pequenos jardins"],
        "documentários": ["assinatura de streaming com foco em documentários (ex: CuriosityStream, Nebula)", "biografia de uma figura histórica ou científica interessante"],
        "caminhada": ["botas de trekking de alta qualidade e impermeáveis", "mochila leve e ergonômica para caminhada", "bastão de caminhada dobrável e ajustável"],
        "arte digital": ["mesa digitalizadora com mais níveis de pressão", "software de ilustração profissional (assinatura ou licença)", "curso online avançado de concept art ou pintura digital"],
        "música indie": ["box especial de vinis de uma banda favorita", "fone de ouvido de alta fidelidade para audiófilos", "ingresso para um show ou festival de música indie"],
        "gatos": ["arranhador grande e interativo tipo árvore para gatos", "câmera interativa para pets com dispensador de petiscos", "assinatura de uma caixa mensal de brinquedos e petiscos para gatos"],
    }
    # Retorna a lista de presentes para o interesse, ou uma lista padrão se o interesse não estiver no catálogo.
    return catalogo.get(interesse_chave.lower(), ["vale-presente personalizado e criativo", "uma experiência única e memorável (ex: aula de culinária, voo de balão, dia de spa)"])

def sugerir_presentes(nome, interesses, restricoes, dias_para_aniversario):
    """Gera sugestões de presentes com o modelo Gemini Flash, considerando interesses, restrições e urgência."""
    if not model: return "Modelo Gemini não configurado (verifique API Key e Célula 1)."

    # Coleta sugestões do catálogo simulado.
    sug_cat = list(set(s for i in interesses if i for s in simular_catalogo_presentes(i))) # Remove duplicatas

    # Adapta o prompt com base na urgência do aniversário.
    urg_txt = ""
    if dias_para_aniversario == 0:
        urg_txt = "O aniversário é HOJE! As sugestões devem ser extremamente práticas e rápidas de conseguir (ex: vale-presente digital enviado por email, algo que possa ser comprado em uma loja física próxima rapidamente, uma experiência agendável para hoje ou amanhã)."
    elif dias_para_aniversario <= 2: # Se faltam 1 ou 2 dias.
        urg_txt = f"O aniversário é daqui a apenas {dias_para_aniversario} dia(s)! As sugestões devem priorizar ideias que possam ser compradas ou entregues rapidamente (ex: entrega expressa, produtos digitais)."

    # Cria o prompt detalhado para o modelo de IA.
    prompt = f"""Você é um assistente especialista em sugerir presentes criativos, atenciosos e práticos.
Preciso de 3 a 4 ideias de presentes para {nome}.
{urg_txt}

Perfil da pessoa:
- Interesses principais de {nome}: {', '.join(interesses) if interesses else 'Não foram informados muitos detalhes sobre os interesses.'}.
- Restrições importantes (principalmente alimentares ou alergias): {', '.join(restricoes) if restricoes else 'Nenhuma restrição informada.'}.

Sugestões iniciais do nosso catálogo (use como inspiração, mas sinta-se à vontade para ir além, especialmente se as sugestões do catálogo não forem práticas dada a urgência):
{', '.join(sug_cat) if sug_cat else 'Nenhuma sugestão específica do catálogo para estes interesses.'}.

Instruções para as ideias de presentes:
1.  **Criatividade e Personalização:** Devem parecer pensadas especialmente para {nome}.
2.  **Relevância aos Interesses:** Devem, sempre que possível, conectar-se aos interesses listados.
3.  **Respeito às Restrições:** ESTE É O MAIS IMPORTANTE. Todas as sugestões devem respeitar integralmente as restrições. Se uma sugestão puder, mesmo que remotamente, conflitar com uma restrição (ex: cesta de café da manhã para alguém com restrição a glúten), adicione um ALERTA MUITO CLARO e sugira como adaptar (ex: "Cesta de café da manhã, MAS CERTIFIQUE-SE que todos os itens sejam sem glúten e sem contaminação cruzada.").
4.  **Especificidade e Utilidade:** Dê ideias concretas, não genéricas.
5.  **Adequação à Urgência:** Considere o quão próximo está o aniversário ao sugerir (ex: presentes digitais para aniversários "em cima da hora").

Formato da resposta:
Liste as ideias de presente em tópicos (usando '*' ou '-').
Para cada ideia, forneça uma breve justificativa do porquê seria um bom presente para {nome} e como ela se adequa à urgência (se aplicável)."""
    try:
        # Envia o prompt para o modelo Gemini e retorna o texto da resposta.
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        return f"Erro ao gerar sugestões de presentes com Gemini: {e}"

# --- ORQUESTRAÇÃO: O "Maestro" que coordena os agentes ---
def orquestrar_aniversario_inteligente():
    """Função principal que executa todo o fluxo do programa Aniversariômetro."""

    # Verifica se as configurações da Célula 1 (Drive e Caminhos) estão OK.
    if 'drive_montado_com_sucesso_global' not in globals() or not drive_montado_com_sucesso_global:
        print("🔴🔴 ERRO CRÍTICO: Google Drive não está acessível (problema detectado na CÉLULA 1).")
        print("     O Aniversariômetro não pode funcionar corretamente sem acesso ao Google Drive.")
        print("     Por favor, resolva o problema na CÉLULA 1 (montagem do Drive, permissões) e reinicie esta célula (CÉLULA 2).")
        return # Interrompe a execução da função.
    # Verifica se o modelo Gemini foi carregado.
    if not model:
        print("🔴🔴 ERRO CRÍTICO: Modelo Gemini não carregado (provavelmente problema com API Key ou configuração inicial).")
        print("     Verifique a configuração da sua API Key nos Secrets do Colab e se não houve erros no início da CÉLULA 2.")
        return # Interrompe a execução da função.

    print(f"🤖 Bem-vindo ao Aniversariômetro Inteligente (Drive Exclusivo + Gemini Flash)! 🥳")
    # Informa ao usuário onde os dados estão sendo gerenciados (caminho definido na Célula 1).
    if NOME_ARQUIVO_DADOS_GLOBAL:
        print(f"   Os dados dos aniversários serão lidos e salvos em: {NOME_ARQUIVO_DADOS_GLOBAL}")
    print("---")

    # Loop para permitir o cadastro de um ou mais aniversariantes.
    while True:
        print("\n--- CADASTRO DE ANIVERSARIANTE ---")
        resp_cad = input("Deseja cadastrar um novo aniversariante? (s/n, ou 'pular' para finalizar cadastros): ").lower()
        if resp_cad in ['n', 'pular']:
            break # Sai do loop de cadastro.
        if resp_cad == 's':
            nome = input("Nome do aniversariante: ").strip() # .strip() remove espaços extras no início/fim.
            if not nome: # Verifica se o nome não está vazio.
                print("Nome não pode ser vazio. Tente novamente.")
                continue # Volta para o início do loop de cadastro.
            # Loop para garantir que a data de nascimento seja inserida no formato correto.
            while True:
                data_nasc = input(f"Data de nascimento de {nome} (formato AAAA-MM-DD, ex: 1990-07-25): ").strip()
                try:
                    datetime.strptime(data_nasc, "%Y-%m-%d") # Apenas valida o formato.
                    break # Sai do loop da data se o formato for válido.
                except ValueError:
                    print("Formato de data inválido. Por favor, use AAAA-MM-DD (ano com 4 dígitos, mês com 2, dia com 2).")
            # Coleta interesses.
            interesses_str = input(f"Interesses de {nome} (ex: leitura, viagens, café). Separe por vírgulas, ou deixe em branco se não souber: ").strip()
            # Converte a string de interesses em uma lista, removendo itens vazios ou com apenas espaços.
            interesses = [i.strip() for i in interesses_str.split(',') if i.strip()]
            # Coleta restrições.
            restricoes_str = input(f"{nome} possui alguma restrição importante (ex: sem glúten, alergia a nozes)? Separe por vírgulas, ou deixe em branco se não houver: ").strip()
            restricoes = [r.strip() for r in restricoes_str.split(',') if r.strip()]
            # Coleta o relacionamento.
            relac = input(f"Qual é o seu relacionamento com {nome}? (ex: amigo(a), colega de trabalho, mãe, primo). Isso ajuda a personalizar as mensagens: ").strip()
            # Se o usuário não digitar nada para relacionamento, usa um valor padrão.
            if not relac:
                relac = "pessoa especial"
            # Chama a função para guardar o aniversário.
            guardar_aniversario(nome, data_nasc, interesses, restricoes, relac)
        else:
            print("Opção inválida. Por favor, digite 's' para sim, 'n' para não, ou 'pular'.")

    # --- Seção de Depuração Global (comentada na versão final) ---
    # Se precisar investigar o conteúdo da lista 'aniversarios_cadastrados' antes da verificação,
    # descomente a linha abaixo. Ela é muito útil para depuração.
    # print("\n[DEPURAÇÃO GLOBAL] Estado de aniversarios_cadastrados ANTES da verificação:", aniversarios_cadastrados)

    print("\n--- VERIFICANDO ANIVERSÁRIOS PRÓXIMOS ---")
    dias_ch = 7 # Define o período de antecedência para os alertas (hoje e os próximos 7 dias).
    aniv_prox = verificar_aniversarios_proximos(dias_antecedencia=dias_ch)

    if aniv_prox: # Se a lista de aniversários próximos não estiver vazia.
        print(f"\n✨ Temos {len(aniv_prox)} aniversário(s) importante(s) (hoje ou nos próximos {dias_ch} dias)! ✨")
        for aniv in aniv_prox: # Itera sobre cada aniversariante próximo.
            # Formata a data de nascimento para mostrar apenas dia/mês.
            dt_fmt = datetime.strptime(aniv['data_nasc'], "%Y-%m-%d").strftime("%d/%m")
            d_falta = aniv['dias_para_aniversario'] # Pega os dias que faltam.
            # Define a mensagem de "quando é" o aniversário.
            qdo = f"🎉 HOJE, {dt_fmt} 🎉" if d_falta == 0 else \
                  (f"🗓️ Amanhã, {dt_fmt}!" if d_falta == 1 else \
                   f"🗓️ Em {d_falta} dias ({dt_fmt})")
            print(f"\n👉 Aniversário de {aniv['nome'].upper()} é {qdo}!") # Nome em maiúsculas para destaque.

            # Gera e imprime a mensagem de parabéns.
            print(f"\n----- 💌 Mensagem Sugerida para {aniv['nome']} (via Gemini Flash) -----")
            print(gerar_mensagem_parabens(aniv['nome'], aniv['relacionamento'], d_falta))
            print("---------------------------------------")

            # Gera e imprime as sugestões de presentes.
            print(f"\n----- 🎁 Ideias de Presente para {aniv['nome']} (via Gemini Flash) -----")
            print(sugerir_presentes(aniv['nome'], aniv['interesses'], aniv['restricoes'], d_falta))
            print("---------------------------------------")
    else: # Se não houver aniversários próximos.
        print(f"\nNenhum aniversário hoje ou nos próximos {dias_ch} dias. Mas fique de olho! 👀")
    print("\nObrigado por usar o Aniversariômetro Inteligente! 🥳")

# --- Execução Principal ---
# Este bloco de código só é executado quando o script é rodado diretamente
# (e não quando é importado como um módulo em outro script).
if __name__ == "__main__":
    # Verifica se as variáveis globais da Célula 1 existem e se o Drive foi montado
    # antes de chamar a função principal do programa.
    if 'drive_montado_com_sucesso_global' in globals() and drive_montado_com_sucesso_global and \
       'NOME_ARQUIVO_DADOS_GLOBAL' in globals() and NOME_ARQUIVO_DADOS_GLOBAL:
        orquestrar_aniversario_inteligente()
    else:
        # Mensagem de erro se a Célula 1 não foi executada corretamente.
        print("🔴 ERRO FATAL: O Aniversariômetro não pode iniciar.")
        print("   Certifique-se de que a CÉLULA 1 (montagem do Drive e configuração de caminhos) foi executada com sucesso ANTES desta célula.")

📦 Instalando/verificando biblioteca google-generativeai...
   Biblioteca google-generativeai pronta.
🔑 API Key do Gemini configurada e modelo 'gemini-1.5-flash-latest' carregado!
ℹ️ Arquivo '/content/drive/MyDrive/AniversariometroApp_Dados_Final/aniversarios_gdrive_final.json' não encontrado. Criando com 1 exemplo.
🤖 Bem-vindo ao Aniversariômetro Inteligente (Drive Exclusivo + Gemini Flash)! 🥳
   Os dados dos aniversários serão lidos e salvos em: /content/drive/MyDrive/AniversariometroApp_Dados_Final/aniversarios_gdrive_final.json
---

--- CADASTRO DE ANIVERSARIANTE ---
Deseja cadastrar um novo aniversariante? (s/n, ou 'pular' para finalizar cadastros): n

--- VERIFICANDO ANIVERSÁRIOS PRÓXIMOS ---

Nenhum aniversário hoje ou nos próximos 7 dias. Mas fique de olho! 👀

Obrigado por usar o Aniversariômetro Inteligente! 🥳
